In [1]:
import requests
import json
import datetime
from dateutil.parser import parse as dp
import pymysql
import time
import pandas as pd
import uuid
import sys

In [2]:
web_id = {'92788076': {'brand': '亚特兰蒂斯',
                       'shopName': '三亚亚特兰蒂斯酒店',
                       'url': "http://hotel.elong.com/92788076/?issugtrace=2#review"},
         
        '52021013': {'brand': '爱必侬',
                        'shopName': '恩平爱必侬泉林酒店',
                        'url': "http://hotel.elong.com/52021013/?issugtrace=2#review"},
                   
        '63873449': {'brand': '爱必侬',
                        'shopName': '金佛山爱必侬度假村',
                        'url': "http://hotel.elong.com/63873449/?issugtrace=2#review"},
          
        '93557150': {'brand': '爱必侬',
                        'shopName': '爱必侬棠湾度假公寓(三亚亚特兰蒂斯度假区店)',
                        'url': "http://hotel.elong.com/93557150/?issugtrace=2#review"}
         }

In [3]:
#COOKIES = 'CookieGuid=62dcb3f9-4ed9-46a0-865c-38f4983f535d; firsttime=1621838677310; H5CookieId=c3470134-0b6a-4086-9e4f-8b0873d948a8; CitySearchHistory=0101%23%E5%8C%97%E4%BA%AC%23beijing%23; SHBrowseHotel=cn=92788076%2C%2C%2C%2C%2C%2C%3B93557150%2C%2C%2C%2C%2C%2C%3B52021013%2C%2C%2C%2C%2C%2C%3B63873449%2C%2C%2C%2C%2C%2C%3B&; ShHotel=InDate=2021-06-23&CityID=0101&CityNameEN=beijing&CityNameCN=%E5%8C%97%E4%BA%AC&OutDate=2021-06-24&CityName=%E5%8C%97%E4%BA%AC; lasttime=1624416047110; ext_param=bns%3D4%26ct%3D3; businessLine=hotel; H5Channel=mnoreferseo%2CSEO; indate=2021-06-24; outdate=2021-06-25; searchEntranceId=h5_home; DetailNewOrOld=newDetail; SessionGuid=f09cb279-a8ed-41b3-b9d1-28834a552c29; Esid=6c310020-1c2f-47b1-8e1d-c67535330c89; com.eLong.CommonService.OrderFromCookieInfo=Orderfromtype=1&Parentid=50000&Status=1&Cookiesdays=0&Coefficient=0.0&Pkid=50&Priority=8000&Isusefparam=0&Makecomefrom=0&Savecookies=0; fv=pcweb; JSESSIONID=080DA6B595F4F025C34642011BCE50AF'
#     'Referer': 'http://hotel.elong.com/hotel/hoteldetail?issugtrace=2&hotelId=92788076&inDate=2021-06-24&outDate=2021-06-25',
HEADERS = {
    'Accept-Language': 'application/json, text/plain, */*',
    'Connection': 'keep-alive',
    'X-Requested-With': 'XMLHttpRequest',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'hotel.elong.com',
    'Origin': 'http://hotel.elong.com',

    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    
}
#'ookieC': COOKIES



In [4]:
comment_url = 'http://hotel.elong.com/hotelapi/getCommentList'

page = True
page_index=1
check_time = 0
while page == True:
    
    print(page_index)
    form = {"objectId":"92788076",
            "searchFeatures":
            [{"typeId":1004,"filterIds":["-1"]},
            {"typeId":1002,"filterIds":["-1"]}],
            "keyword":"",
            "pageIndex":page_index,
            "pageSize":10,
            "sortingInfo":{"sortingMethod":1,"sortingDirection":1}
           }
    
    res = requests.post(comment_url,json=form,headers=HEADERS)
    data = res.json()
    
    
    db2 = pymysql.connect(host='49.235.85.215', user='root', password='Ipsos11!', port=3306, db='fuxingData')
    cursor2 = db2.cursor()
    
    siteName='艺龙网'
    ObjectId = form['objectId']
    url = web_id[str(ObjectId)]['url']
    brand = web_id[str(ObjectId)]['brand']
    shopName = web_id[str(ObjectId)]['shopName']
    
    
    
    for i in data['data']['comments']:
        import uuid

        publishTime = datetime.datetime.fromtimestamp(int(i['createTime'])/1000).strftime('%Y-%m-%d %H:%M:%S')


        author = i['commentUser']['nickName']

        score = str(i['commentScore'])
        content = i['content']

        crawlTime=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        updateTime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        id = ('OTA' + crawlTime + '艺龙网' + author + url + content[:50])
        uuid = (uuid.uuid3(uuid.NAMESPACE_OID,str(id)))

        # 12  yilong
        startDate = '2021-08-04'

        if publishTime> startDate: 
            
            print(publishTime)
            check_time = 0
        
            sql = "replace into rawDataOta(uuid,siteName,brand,shopName,url,author,score,content,publishTime,crawlTime,updateTime,id) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) "
            values=(str(uuid),siteName,brand,shopName,url,author,score,content,publishTime,crawlTime,updateTime,id)
            cursor2.execute(sql,values)
            db2.commit()
    
        else:
            check_time += 1
            
        if check_time == 30:
            page = False
            sys.exit()
            
        

    
    page_index+=1
    time.sleep(5.893)


1
2021-08-05 02:17:11
2021-08-05 08:55:58
2021-08-05 03:48:32
2021-08-05 09:51:44
2021-08-04 08:58:53
2021-08-04 02:32:41
2021-08-04 16:36:19
2021-08-04 02:29:00
2021-08-04 06:14:02
2021-08-05 02:03:31
2
2021-08-05 08:39:51
2021-08-04 02:26:18
2021-08-04 03:06:50
2021-08-04 04:48:42
2021-08-05 01:45:01
2021-08-05 06:37:06
2021-08-04 03:29:30
2021-08-04 02:14:32
2021-08-05 02:25:52
2021-08-04 21:14:29
3
2021-08-04 21:14:29
2021-08-04 09:55:20
2021-08-05 11:30:48
2021-08-04 05:36:10
2021-08-04 03:47:11
2021-08-04 03:03:19
2021-08-05 01:17:18
2021-08-04 05:06:39
2021-08-04 02:02:23
2021-08-05 11:29:08
4
2021-08-04 08:28:11
2021-08-05 00:32:38
2021-08-05 04:14:41
2021-08-05 07:53:45
2021-08-04 08:36:17
2021-08-05 05:23:35
5
2021-08-04 06:55:36
2021-08-04 09:57:29
2021-08-05 11:00:22
2021-08-04 05:13:09
6
2021-08-05 03:25:53
2021-08-05 03:50:59
2021-08-04 09:09:31
7
2021-08-04 09:01:35
2021-08-04 02:20:55
8
2021-08-04 08:18:16
2021-08-04 04:58:05
2021-08-04 10:22:16
2021-08-04 01:52:09
9
20

SystemExit: 

d:\python\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
